In [ ]:
from parcels import FieldSet


filenames = {
    "U": "spring.nc",
    "V": "spring.nc",
}

variables = {
    "U": "water_u",
    "V": "water_v",
}

dimensions = {
    "lat": "lat",
    "lon": "lon",
#     "time": "time",
}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

In [ ]:
import cartopy.crs as ccrs
from datetime import timedelta
from parcels import ParticleSet, JITParticle

npart = 1000 # nº de particulas liberadas
lon = [-46] * npart # lon de liberacao das particulas
lat = [-26.83] * npart # lat da liberacao das particulas
repeatdt = timedelta(hours=24)

pset = ParticleSet(
    fieldset=fieldset,
    pclass=JITParticle,
    lon=lon,
    lat=lat,
    repeatdt=repeatdt,
)

domain = {"N": -32.8, "S": -24, "E": -44, "W": -52.8}
kw = {"projection": ccrs.PlateCarree()}
pset.show(domain=None, **kw)

In [ ]:
from parcels import AdvectionRK4
from parcels.tools.statuscodes import ErrorCode


def DeleteParticle(particle, fieldset, time):
    particle.delete()

    
output_file = pset.ParticleFile(
    name="results.nc",
    outputdt=timedelta(hours=24)
)

kernels = pset.Kernel(AdvectionRK4)
pset.execute(
    kernels,
    runtime=timedelta(days=30),
    dt=timedelta(hours=5),
    output_file=output_file,
    recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
)

pset.show(domain=domain, **kw)